# 锂电池生产温度测量

## 导入所需要的库

In [1]:
import pandas as pd # 用于处理数据的工具
import lightgbm as lgb # 机器学习模型 LightGBM
from sklearn.metrics import mean_absolute_error # 评分 MAE 的计算函数
from sklearn.model_selection import train_test_split # 拆分训练集与验证集工具
from tqdm import tqdm # 显示循环的进度条工具
import warnings
warnings.filterwarnings("ignore")

## 数据准备

In [2]:
train_dataset = pd.read_csv("./data/train.csv") # 原始训练数据。
test_dataset = pd.read_csv("./data/test.csv") # 原始测试数据（用于提交）。

submit = pd.DataFrame() # 定义提交的最终数据。
submit["序号"] = test_dataset["序号"] # 对齐测试数据的序号。

MAE_scores = dict() # 定义评分项。

## 观察数据

In [3]:
train_dataset.head()

,序号,时间,流量1,流量2,流量3,流量4,流量5,流量6,流量7,流量8,...,下部温度8,下部温度9,下部温度10,下部温度11,下部温度12,下部温度13,下部温度14,下部温度15,下部温度16,下部温度17
0,1,2022/11/6 9:08,35.668999,36.146000,25.558001,26.195000,25.670000,15.702,16.690001,15.991,...,827,827,827,827,827,827,827,827,827,750
1,2,2022/11/6 9:09,35.995998,36.347000,25.382000,26.348000,26.131001,15.523,16.825001,15.871,...,827,827,827,827,827,827,827,827,827,750
2,3,2022/11/6 9:11,35.340000,36.311001,25.469999,26.093000,25.639000,15.564,15.564000,15.947,...,827,827,827,827,827,827,827,827,827,750
3,4,2022/11/6 9:12,35.585999,36.091000,25.250000,26.127001,25.670000,15.575,16.775999,15.936,...,827,827,827,827,827,827,827,827,827,750
4,5,2022/11/6 9:13,35.946999,36.256001,25.163000,26.399000,25.837999,15.460,16.580999,15.795,...,827,827,827,827,827,827,827,827,827,750


In [4]:
train_dataset.columns

Index(['序号', '时间', '流量1', '流量2', '流量3', '流量4', '流量5', '流量6', '流量7', '流量8',
       '流量9', '流量10', '流量11', '流量12', '流量13', '流量14', '流量15', '流量16', '流量17',
       '上部温度设定1', '上部温度设定2', '上部温度设定3', '上部温度设定4', '上部温度设定5', '上部温度设定6',
       '上部温度设定7', '上部温度设定8', '上部温度设定9', '上部温度设定10', '上部温度设定11', '上部温度设定12',
       '上部温度设定13', '上部温度设定14', '上部温度设定15', '上部温度设定16', '上部温度设定17', '下部温度设定1',
       '下部温度设定2', '下部温度设定3', '下部温度设定4', '下部温度设定5', '下部温度设定6', '下部温度设定7',
       '下部温度设定8', '下部温度设定9', '下部温度设定10', '下部温度设定11', '下部温度设定12', '下部温度设定13',
       '下部温度设定14', '下部温度设定15', '下部温度设定16', '下部温度设定17', '上部温度1', '上部温度2',
       '上部温度3', '上部温度4', '上部温度5', '上部温度6', '上部温度7', '上部温度8', '上部温度9', '上部温度10',
       '上部温度11', '上部温度12', '上部温度13', '上部温度14', '上部温度15', '上部温度16', '上部温度17',
       '下部温度1', '下部温度2', '下部温度3', '下部温度4', '下部温度5', '下部温度6', '下部温度7', '下部温度8',
       '下部温度9', '下部温度10', '下部温度11', '下部温度12', '下部温度13', '下部温度14', '下部温度15',
       '下部温度16', '下部温度17'],
      dtype='object')

In [5]:
test_dataset.head()

,序号,时间,流量1,流量2,流量3,流量4,流量5,流量6,流量7,流量8,...,下部温度设定8,下部温度设定9,下部温度设定10,下部温度设定11,下部温度设定12,下部温度设定13,下部温度设定14,下部温度设定15,下部温度设定16,下部温度设定17
0,1,2023/3/1 4:15,24.222000,24.070000,28.889000,25.264999,26.827999,4.354,21.784,24.323000,...,837,837,837,837,837,837,837,837,837,750
1,2,2023/3/1 4:20,24.138000,23.974001,28.798000,25.517000,27.108999,4.377,21.886,24.384001,...,837,837,837,837,837,837,837,837,837,750
2,3,2023/3/1 4:26,24.152000,24.006001,28.827999,25.114000,26.719999,4.293,21.611,24.099001,...,837,837,837,837,837,837,837,837,837,750
3,4,2023/3/1 4:31,24.108999,23.974001,28.783001,25.114000,27.047001,4.354,21.857,24.337999,...,837,837,837,837,837,837,837,837,837,750
4,5,2023/3/1 4:36,24.350000,24.215000,29.146999,25.416000,25.416000,4.377,21.900,24.444000,...,837,837,837,837,837,837,837,837,837,750


In [6]:
test_dataset.columns

Index(['序号', '时间', '流量1', '流量2', '流量3', '流量4', '流量5', '流量6', '流量7', '流量8',
       '流量9', '流量10', '流量11', '流量12', '流量13', '流量14', '流量15', '流量16', '流量17',
       '上部温度设定1', '上部温度设定2', '上部温度设定3', '上部温度设定4', '上部温度设定5', '上部温度设定6',
       '上部温度设定7', '上部温度设定8', '上部温度设定9', '上部温度设定10', '上部温度设定11', '上部温度设定12',
       '上部温度设定13', '上部温度设定14', '上部温度设定15', '上部温度设定16', '上部温度设定17', '下部温度设定1',
       '下部温度设定2', '下部温度设定3', '下部温度设定4', '下部温度设定5', '下部温度设定6', '下部温度设定7',
       '下部温度设定8', '下部温度设定9', '下部温度设定10', '下部温度设定11', '下部温度设定12', '下部温度设定13',
       '下部温度设定14', '下部温度设定15', '下部温度设定16', '下部温度设定17'],
      dtype='object')

In [7]:
submit.head()

,序号
0,1
1,2
2,3
3,4
4,5


## 设定模型训练参数

In [8]:
# 参数设置
pred_labels = list(train_dataset.columns[-34:]) # 需要预测的标签。
train_set, valid_set = train_test_split(train_dataset, test_size=0.2) # 拆分数据集。

# 设定 LightGBM 训练参，查阅参数意义：https://lightgbm.readthedocs.io/en/latest/Parameters.html
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'min_child_weight': 5,
        'num_leaves': 2 ** 5,
        'lambda_l2': 10,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 4,
        'learning_rate': 0.05,
        'seed': 2023,
        'nthread' : 16,
        'verbose' : -1,
    }

no_info = lgb.callback.log_evaluation(period=-1) # 禁用训练日志输出。

In [9]:
pred_labels

['上部温度1',
 '上部温度2',
 '上部温度3',
 '上部温度4',
 '上部温度5',
 '上部温度6',
 '上部温度7',
 '上部温度8',
 '上部温度9',
 '上部温度10',
 '上部温度11',
 '上部温度12',
 '上部温度13',
 '上部温度14',
 '上部温度15',
 '上部温度16',
 '上部温度17',
 '下部温度1',
 '下部温度2',
 '下部温度3',
 '下部温度4',
 '下部温度5',
 '下部温度6',
 '下部温度7',
 '下部温度8',
 '下部温度9',
 '下部温度10',
 '下部温度11',
 '下部温度12',
 '下部温度13',
 '下部温度14',
 '下部温度15',
 '下部温度16',
 '下部温度17']

## 提取时间特征

In [10]:
# 时间特征函数
def time_feature(data: pd.DataFrame, pred_labels: list=None) -> pd.DataFrame:
    """提取数据中的时间特征。

    输入: 
        data: Pandas.DataFrame
            需要提取时间特征的数据。

        pred_labels: list, 默认值: None
            需要预测的标签的列表。如果是测试集，不需要填入。
    
    输出: data: Pandas.DataFrame
            提取时间特征后的数据。
    """
    
    data = data.copy() # 复制数据，避免后续影响原始数据。
    data = data.drop(columns=["序号"]) # 去掉”序号“特征。
    
    data["时间"] = pd.to_datetime(data["时间"]) # 将”时间“特征的文本内容转换为 Pandas 可处理的格式。
    # print(data['时间'])
    data["month"] = data["时间"].dt.month # 添加新特征“month”，代表”当前月份“。
    data["day"] = data["时间"].dt.day # 添加新特征“day”，代表”当前日期“。
    data["hour"] = data["时间"].dt.hour # 添加新特征“hour”，代表”当前小时“。
    data["minute"] = data["时间"].dt.minute # 添加新特征“minute”，代表”当前分钟“。
    data["weekofyear"] = data["时间"].dt.isocalendar().week.astype(int) # 添加新特征“weekofyear”，代表”当年第几周“，并转换成 int，否则 LightGBM 无法处理。
    data["dayofyear"] = data["时间"].dt.dayofyear # 添加新特征“dayofyear”，代表”当年第几日“。
    data["dayofweek"] = data["时间"].dt.dayofweek # 添加新特征“dayofweek”，代表”当周第几日“。
    data["is_weekend"] = data["时间"].dt.dayofweek // 6 # 添加新特征“is_weekend”，代表”是否是周末“，1 代表是周末，0 代表不是周末。

    data = data.drop(columns=["时间"]) # LightGBM 无法处理这个特征，它已体现在其他特征中，故丢弃。

    if pred_labels: # 如果提供了 pred_labels 参数，则执行该代码块。
        data = data.drop(columns=[*pred_labels]) # 去掉所有待预测的标签。
    
    return data # 返回最后处理的数据。

test_features = time_feature(test_dataset) # 处理测试集的时间特征，无需 pred_labels。
test_features.head(5)

,流量1,流量2,流量3,流量4,流量5,流量6,流量7,流量8,流量9,流量10,...,下部温度设定16,下部温度设定17,month,day,hour,minute,weekofyear,dayofyear,dayofweek,is_weekend
0,24.222000,24.070000,28.889000,25.264999,26.827999,4.354,21.784,24.323000,16.721001,18.652000,...,837,750,3,1,4,15,9,60,2,0
1,24.138000,23.974001,28.798000,25.517000,27.108999,4.377,21.886,24.384001,16.709999,18.600000,...,837,750,3,1,4,20,9,60,2,0
2,24.152000,24.006001,28.827999,25.114000,26.719999,4.293,21.611,24.099001,16.591999,18.471001,...,837,750,3,1,4,26,9,60,2,0
3,24.108999,23.974001,28.783001,25.114000,27.047001,4.354,21.857,24.337999,16.677999,18.561001,...,837,750,3,1,4,31,9,60,2,0
4,24.350000,24.215000,29.146999,25.416000,25.416000,4.377,21.900,24.444000,16.775999,18.704000,...,837,750,3,1,4,36,9,60,2,0


## 模型训练与预测

In [11]:
import matplotlib.pyplot as plt

# 创建一个空列表来保存每次迭代的训练损失
training_loss = []

# 创建一个回调函数来记录每次迭代的训练损失
def record_loss(env):
    training_loss.append(env.evaluation_result_list[0][2])

# 从所有待预测特征中依次取出每类的标签进行训练与预测。
for pred_label in tqdm(pred_labels):
    print("当前的pred_label是：", pred_label)
    train_features = time_feature(train_dataset, pred_labels=pred_labels) # 处理训练集的时间特征。
    train_labels = train_dataset[pred_label] # 训练集的对应类别的标签数据。

    train_data = lgb.Dataset(train_features, label=train_labels) # 将训练集转换为 LightGBM 可处理的类型。

    valid_features = time_feature(valid_set, pred_labels=pred_labels) # 处理验证集的时间特征。
    # valid_features = enhancement(valid_features_raw)
    valid_labels = valid_set[pred_label] # 验证集的标签数据。
    # print("当前的valid_labels是：", valid_labels)
    valid_data = lgb.Dataset(valid_features, label=valid_labels) # 将验证集转换为 LightGBM 可处理的类型。

    # 训练模型，参数依次为：导入模型设定参数、导入训练集、设定模型迭代次数（200）、导入验证集、禁止输出日志
    model = lgb.train(lgb_params, train_data, 200, valid_sets=valid_data)
    
    valid_pred = model.predict(valid_features, num_iteration=model.best_iteration) # 选择效果最好的模型进行验证集预测。
    test_pred = model.predict(test_features, num_iteration=model.best_iteration) # 选择效果最好的模型进行测试集预测。
    MAE_score = mean_absolute_error(valid_pred, valid_labels) # 计算验证集预测数据与真实数据的 MAE。
    MAE_scores[pred_label] = MAE_score # 将对应标签的 MAE 值 存入评分项中。

    submit[pred_label] = test_pred # 将测试集预测数据存入最终提交数据中。
    # 创建一个图表来显示训练损失随迭代次数的变化
    # plt.plot(training_loss)
    # plt.xlabel('Iteration')
    # plt.ylabel('Training Loss')
    # plt.show()


  0%|          | 0/34 [00:00<?, ?it/s]

当前的pred_label是： 上部温度1


  3%|▎         | 1/34 [00:00<00:16,  2.02it/s]

当前的pred_label是： 上部温度2


  6%|▌         | 2/34 [00:01<00:16,  1.99it/s]

...

 94%|█████████▍| 32/34 [00:16<00:01,  1.94it/s]

当前的pred_label是： 下部温度16


 97%|█████████▋| 33/34 [00:16<00:00,  1.97it/s]

当前的pred_label是： 下部温度17


100%|██████████| 34/34 [00:17<00:00,  1.97it/s]


## 保存结果

In [12]:
submit.to_csv('./data/submit_result.csv', index=False) # 保存最后的预测结果到 submit_result.csv
print(MAE_scores) # 查看各项的 MAE 值。

{'上部温度1': 5.040988298006909, '上部温度2': 4.652432987059698, '上部温度3': 3.2484328268791733, '上部温度4': 1.9507754913465625, '上部温度5': 2.4572380717209996, '上部温度6': 2.074353987517434, '上部温度7': 1.389370177963195, '上部温度8': 0.11827481019292359, '上部温度9': 0.048430293848361096, '上部温度10': 0.04478503801364404, '上部温度11': 0.060027147167978426, '上部温度12': 0.07481352527710482, '上部温度13': 0.06406748829523433, '上部温度14': 0.09462945258791236, '上部温度15': 0.17201355792732875, '上部温度16': 0.24727032766737067, '上部温度17': 0.8023878182592751, '下部温度1': 2.937577001772735, '下部温度2': 1.2501543112624107, '下部温度3': 0.5500564107613642, '下部温度4': 0.27037337091720565, '下部温度5': 0.8984951556591149, '下部温度6': 0.543050607816922, '下部温度7': 0.2138228780966383, '下部温度8': 0.050087959904197316, '下部温度9': 4.595571421605266, '下部温度10': 0.04739709682448464, '下部温度11': 0.08336675401568777, '下部温度12': 0.09402325673064997, '下部温度13': 0.07383412085542158, '下部温度14': 0.07959783779164084, '下部温度15': 0.1360256457867532, '下部温度16': 0.1948130259736614, '下部温度17': 0.249

# 一些自己的修改

## 交叉验证

In [13]:
# 参数设置
pred_labels = list(train_dataset.columns[-34:]) # 需要预测的标签。

# 设定 LightGBM 训练参，查阅参数意义：https://lightgbm.readthedocs.io/en/latest/Parameters.html
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'min_child_weight': 5,
        'num_leaves': 2 ** 5,
        'lambda_l2': 10,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 4,
        'learning_rate': 0.05,
        'seed': 2023,
        'nthread' : 16,
        'verbose' : -1,
    }

no_info = lgb.callback.log_evaluation(period=-1) # 禁用训练日志输出。

In [18]:
from sklearn.model_selection import KFold  
from sklearn.model_selection import cross_val_score
import numpy as np
# Generate the 10-fold partitioning
n_splits = 10 # 10-fold cross-validation
seed = 32 # for reproducibility    
kfold = KFold(n_splits=10, random_state=seed, shuffle=True)  


# Initialize the minimum MAE and the best train/validation split
min_mae = np.inf
best_train = None
best_valid = None

# 从所有待预测特征中依次取出每类的标签进行训练与预测。
for pred_label in tqdm(pred_labels):
    train_features = time_feature(train_set, pred_labels=pred_labels) # 处理训练集的时间特征。
    train_labels = train_set[pred_label] # 训练集的对应类别的标签数据。

    # Perform the 10-fold cross-validation
    for train_index, valid_index in kfold.split(train_features):
        X_train, X_valid = train_features.iloc[train_index], train_features.iloc[valid_index]
        y_train, y_valid = train_labels.iloc[train_index], train_labels.iloc[valid_index]

        # Train the model
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid)
        model = lgb.train(lgb_params, train_data, 200, valid_sets=valid_data)

        # Predict on the validation set and compute the MAE
        valid_pred = model.predict(X_valid, num_iteration=model.best_iteration)
        mae = mean_absolute_error(y_valid, valid_pred)

        # If this MAE is smaller than the current minimum, update the minimum and save this split
        if mae < min_mae:
            min_mae = mae
            best_train = (X_train, y_train)
            best_valid = (X_valid, y_valid)
    train_data = lgb.Dataset(best_train[0], label=best_train[1]) # 将训练集转换为 LightGBM 可处理的类型。
    valid_data = lgb.Dataset(best_valid[0], label=best_valid[1]) # 将验证集转换为 LightGBM 可处理的类型。
# Now, best_train and best_valid contain the training and validation splits that gave the smallest MAE
    
    # 训练模型，参数依次为：导入模型设定参数、导入训练集、设定模型迭代次数（200）、导入验证集、禁止输出日志
    model = lgb.train(lgb_params, train_data, 200, valid_sets=valid_data)
    
    valid_pred = model.predict(best_valid[0], num_iteration=model.best_iteration) # 选择效果最好的模型进行验证集预测。
    test_pred = model.predict(test_features, num_iteration=model.best_iteration) # 选择效果最好的模型进行测试集预测。
    MAE_score = mean_absolute_error(valid_pred, best_valid[1]) # 计算验证集预测数据与真实数据的 MAE。
    MAE_scores[pred_label] = MAE_score # 将对应标签的 MAE 值 存入评分项中。

    submit[pred_label] = test_pred # 将测试集预测数据存入最终提交数据中。

100%|██████████| 34/34 [02:06<00:00,  3.73s/it]


In [19]:
print(MAE_scores)

{'上部温度1': 5.465754055050489, '上部温度2': 4.887864215337955, '上部温度3': 3.4207008894019673, '上部温度4': 2.1205865518411424, '上部温度5': 2.1205865518411424, '上部温度6': 2.1205865518411424, '上部温度7': 1.4807261122203512, '上部温度8': 0.12457070872398143, '上部温度9': 0.06018625401673228, '上部温度10': 0.05596058734554622, '上部温度11': 0.05596058734554622, '上部温度12': 0.05596058734554622, '上部温度13': 0.05596058734554622, '上部温度14': 0.05596058734554622, '上部温度15': 0.05596058734554622, '上部温度16': 0.05596058734554622, '上部温度17': 0.05596058734554622, '下部温度1': 0.05596058734554622, '下部温度2': 0.05596058734554622, '下部温度3': 0.05596058734554622, '下部温度4': 0.05596058734554622, '下部温度5': 0.05596058734554622, '下部温度6': 0.05596058734554622, '下部温度7': 0.05596058734554622, '下部温度8': 0.05596058734554622, '下部温度9': 0.05596058734554622, '下部温度10': 0.05596058734554622, '下部温度11': 0.05596058734554622, '下部温度12': 0.05596058734554622, '下部温度13': 0.05596058734554622, '下部温度14': 0.05596058734554622, '下部温度15': 0.05596058734554622, '下部温度16': 0.05596058734554622, '下部